# Extract FEMA Flood Claim Data
This notebook uses the [FEMA API](https://www.fema.gov/about/openfema/api) to fetch [FIMA NFIP Redacted Claims](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2) data for North Carolina. The results are saved into the `data`/`raw` folder as `NC_Claims.csv`. 

**Sources/Links**: 
* NFIP Redacted Claims: <https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2>
* FEMA API Documentation: <https://www.fema.gov/about/openfema/api>

**Damage codes**:  
0 : Other causes;  
1 : Tidal water overflow;  
2 : Stream, river, or lake overflow;  
3 : Alluvial fan overflow;  
4 : Accumulation of rainfall or snowmelt;  
7 : Erosion-demolition;  
8 : Erosion-removal;  
9 : Earth movement, landslide, land subsidence, sinkholes, etc.;  
A : Closed basin lake;  
B : Expedited claim handling process without site inspection;  
C : Expedited claim handling process follow-up site inspection;  
D : Expedited claim handling process by Adjusting Process Pilot Program (Remote Adjustment); 



In [1]:
#Packages
from pathlib import Path
import pandas as pd
import requests

In [2]:
#Set the state to extract
the_state = 'NC'

In [3]:
#Set fields to extract
fields = [
    'dateOfLoss','occupancyType','amountPaidOnBuildingClaim','totalBuildingInsuranceCoverage',
    'yearOfLoss','buildingDamageAmount','netBuildingPaymentAmount','causeOfDamage','floodEvent',
    'state','reportedZipCode','countyCode','censusTract','censusBlockGroupFips',
    'latitude','longitude']

In [4]:
#Construct the request and send it, coding the response as a json object (dictionary)
base_url = f'https://www.fema.gov/api/open/v2/FimaNfipClaims?$filter=state%20eq%20%27{the_state}%27&&$select={",".join(fields)}&$allrecords=true'
response = requests.get(base_url)
resp_json = response.json()

In [5]:
#Extract claims from the fetched data
claims = resp_json['FimaNfipClaims']
len(claims)

109268

In [10]:
#Convert to a dataframe
df = pd.DataFrame(claims)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109268 entries, 0 to 109267
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   dateOfLoss                      109268 non-null  object 
 1   occupancyType                   109265 non-null  float64
 2   amountPaidOnBuildingClaim       83993 non-null   float64
 3   totalBuildingInsuranceCoverage  109268 non-null  int64  
 4   yearOfLoss                      109268 non-null  int64  
 5   buildingDamageAmount            89218 non-null   float64
 6   netBuildingPaymentAmount        109268 non-null  float64
 7   causeOfDamage                   107407 non-null  object 
 8   floodEvent                      85463 non-null   object 
 9   state                           109268 non-null  object 
 10  reportedZipCode                 109268 non-null  object 
 11  countyCode                      108423 non-null  object 
 12  censusTract     

In [15]:
#Fix occupancyType: remove decimal and save as a string
df['occupancyType'] = df['occupancyType'].fillna(0).astype('int64').astype('str')

In [16]:
#Save locally
df.to_csv(Path.cwd().parent/'data'/'raw'/'NC_Claims.csv',index=False)